In [ ]:
from datetime import datetime

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import quicklook_xarray
from gfatpy.lidar.retrieval.klett import klett_rcs
from gfatpy.atmo.ecmwf import get_ecmwf_day
from gfatpy.utils.utils import moving_average

from gfatpy.utils.calibration import (molecular_properties_2d,
    mask_by_slope,
    mask_by_corrcoef,
    cluster_value,
    cluster_at_least
)

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
date = datetime(2021, 5, 26)
nc = preprocess("mhc_1a_Prs_rs_xf_20210526.nc")

mol_properties = molecular_properties_2d(date, heights=nc.range.values, times=nc.time.values, wavelength=532)
rcs = nc.signal_532xpp * nc.range**2

In [ ]:
nc.signal_532xpa

## Heights calibration

In [ ]:
quicklook_xarray(nc.signal_532xpp, is_rcs = False, scale_bounds=(0, 4e8))
plt.show()

### Slopes masks

In [ ]:
# window 1000
slope_mask = mask_by_slope(rcs, mol_properties.attenuated_molecular_beta, plot_profile = 30)
quicklook_xarray(slope_mask)

In [ ]:
rcs.where(~slope_mask).plot(y = "range", vmin = 0, cmap = "jet", vmax = 1e8)

In [ ]:
# window 500
slope_mask = mask_by_slope(rcs, mol_properties.attenuated_molecular_beta, window_size=500)
quicklook_xarray(slope_mask)

In [ ]:
# window 250
slope_mask = mask_by_slope(rcs, mol_properties.attenuated_molecular_beta, window_size=250)
quicklook_xarray(slope_mask)

### Correlation masks

In [ ]:
slope_mask = mask_by_corrcoef(
    rcs,
    mol_properties.attenuated_molecular_beta,
    window_size = 1000,
    window_time = 15,
    min_corr = 0.95
)

quicklook_xarray(slope_mask, scale_bounds = "limits")

In [ ]:
rcs.where(slope_mask < 0.9).plot(y = "range", vmin = 0, cmap = "jet", vmax = 1e8)

In [ ]:
def smooth_profiles(arr: np.ndarray, /) -> np.ndarray:
    def smooth_profile(_x: np.ndarray) -> np.ndarray:
        return savgol_filter(_x, 150, 3)

    return np.apply_along_axis(smooth_profile, 1, arr)


In [ ]:
rcs_sel = xr.apply_ufunc(
    smooth_profiles, rcs
)
rcs_sel = xr.apply_ufunc(
    moving_average, rcs_sel, kwargs={"window_size": 15}
)

rcs_sel[30].plot()
rcs_sel[30].where(slope_mask[0]).plot()

plt.yscale("log")

In [ ]:
mol_properties.attenuated_molecular_beta

In [ ]:
np.corrcoef([1, 2, 3], [2, 4, 5])

### Klett inversion

In [ ]:
quicklook_xarray(slope_mask)

In [ ]:
clusters = cluster_value(slope_mask.values, value = 1)
mol_idx = cluster_at_least(clusters, 50)

In [ ]:
quicklook_xarray(nc.signal_532xpp, is_rcs = False, scale_bounds=(0, 4e8))

plt.scatter(nc.time, mol_idx[:, 0] * 7.5, 5, c = "w")
plt.scatter(nc.time, mol_idx[:, 1] * 7.5, 5, c = "gold")
plt.show()

In [ ]:
beta_array = xr.full_like(rcs, np.nan)

for time_idx in range(beta_array.time.shape[0]):
    
    cal_limits = mol_idx[time_idx]
    
    if np.isnan(cal_limits).any():
        continue
    
    att_beta = mol_properties.attenuated_molecular_beta[time_idx].values
    
    try:
        ymin, ymax = nc.range[mol_idx[time_idx].astype(int)].values
    except:
        set_trace()
    
    beta = klett_rcs(
        rcs[time_idx].values,
        rcs.range.values,
        att_beta,
        ymin = ymin,
        ymax = ymax
    )
    print(f"Beta computed for profile {time_idx}")
    
    beta_array[time_idx] = beta

In [ ]:
beta_array[:] = np.where(beta_array.values <= 1e-9, np.nan, beta_array.values)

In [ ]:
quicklook_xarray(beta_array, scale_bounds=(0, 3e-6))

In [ ]:
beta_array = xr.full_like(rcs, np.nan)

for time_idx in range(beta_array.time.shape[0]):
    
    cal_limits = mol_idx[time_idx]
    
    if np.isnan(cal_limits).any():
        continue
    
    att_beta = mol_properties.attenuated_molecular_beta[time_idx].values
    
    try:
        ymin, ymax = nc.range[mol_idx[time_idx].astype(int)].values
    except:
        set_trace()
    
    beta = klett_rcs(
        rcs[time_idx].values,
        rcs.range.values,
        att_beta,
        ymin = ymin,
        ymax = ymax
    )
    print(f"Beta computed for profile {time_idx}")
    
    beta_array[time_idx] = beta

In [ ]:
beta_array[:] = np.where(beta_array.values == 0, np.nan, beta_array.values)

In [ ]:
quicklook_xarray(beta_array, scale_bounds=(0, 1.8e-6))